In [49]:
import numpy as np
import gym
from stable_baselines3 import PPO
import math
import itertools
import datetime
import ray
import ray.rllib as rllib
from ray import tune
from ray.tune.registry import register_env

In [2]:
N = 10
rand = np.random.RandomState(1)
M = np.round(rand.normal(5, 1.5, size=(N, N)),2)
M[np.eye(N)==1] = 0
M

array([[0.  , 4.08, 4.21, 3.39, 6.3 , 1.55, 7.62, 3.86, 5.48, 4.63],
       [7.19, 0.  , 4.52, 4.42, 6.7 , 3.35, 4.74, 3.68, 5.06, 5.87],
       [3.35, 6.72, 0.  , 5.75, 6.35, 3.97, 4.82, 3.6 , 4.6 , 5.8 ],
       [3.96, 4.4 , 3.97, 0.  , 3.99, 4.98, 3.32, 5.35, 7.49, 6.11],
       [4.71, 3.67, 3.88, 7.54, 0.  , 4.04, 5.29, 8.15, 5.18, 5.93],
       [5.45, 4.47, 3.29, 4.48, 4.69, 0.  , 6.26, 6.4 , 5.43, 6.33],
       [3.87, 6.88, 5.77, 4.55, 5.73, 4.89, 0.  , 7.28, 8.28, 2.91],
       [2.83, 4.24, 5.24, 6.31, 5.47, 1.97, 4.54, 0.  , 5.35, 6.14],
       [4.67, 4.7 , 5.28, 5.62, 5.3 , 5.18, 3.99, 5.57, 0.  , 6.69],
       [6.8 , 5.28, 4.44, 4.04, 5.64, 5.12, 4.48, 5.07, 4.07, 0.  ]])

In [3]:
math.factorial(N-1)

362880

In [22]:
class MyEnv(gym.Env):
    def __init__(self, env_config):
        super().__init__()
        self.n = N-1
        self.action_space = gym.spaces.Discrete(self.n)
        self.observation_space = gym.spaces.Dict({
            'visited': gym.spaces.MultiBinary(self.n), 
            'last': gym.spaces.Discrete(N)})   
        
    def reset(self):
        self.state = {'visited': np.zeros(self.n), 'last': 0}
        visited = np.zeros(self.n)
        return self.state

    def step(self, action):
        if self.state['visited'][action] == 1:
            self.reward = -10
        else:
            self.state['visited'][action] = 1
            self.reward = - M[self.state['last'], action + 1]
            self.state['last'] = action + 1
        if np.all(self.state['visited'] == 1): 
            self.reward += - M[action + 1, 0]    
            self.done = True
        else:
            self.done = False
            
        return self.state, self.reward, self.done, {}

In [23]:
config = rllib.agents.ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
config["framework"] = "torch"
config["env_config"] = {}

In [24]:
env = MyEnv(config)

In [25]:
print(datetime.datetime.now())
model = PPO("MultiInputPolicy", env, verbose=1)
model.learn(total_timesteps=100000)
print(datetime.datetime.now())

2022-03-13 19:55:25.546275
Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 27.3     |
|    ep_rew_mean     | -233     |
| time/              |          |
|    fps             | 1777     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.2        |
|    ep_rew_mean          | -193        |
| time/                   |             |
|    fps                  | 1378        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.005818229 |
|    clip_fraction        | 0.00898     |
|    clip_range           | 0.2    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12          |
|    ep_rew_mean          | -80.8       |
| time/                   |             |
|    fps                  | 1046        |
|    iterations           | 11          |
|    time_elapsed         | 21          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.018074024 |
|    clip_fraction        | 0.144       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.93       |
|    explained_variance   | -6.2e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 452         |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0253     |
|    value_loss           | 811         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 11      

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.23        |
|    ep_rew_mean          | -49         |
| time/                   |             |
|    fps                  | 1037        |
|    iterations           | 21          |
|    time_elapsed         | 41          |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.014562448 |
|    clip_fraction        | 0.148       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.33       |
|    explained_variance   | -4.53e-06   |
|    learning_rate        | 0.0003      |
|    loss                 | 98.7        |
|    n_updates            | 200         |
|    policy_gradient_loss | -0.0166     |
|    value_loss           | 180         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.18  

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.1         |
|    ep_rew_mean          | -43.9       |
| time/                   |             |
|    fps                  | 1002        |
|    iterations           | 31          |
|    time_elapsed         | 63          |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.009600232 |
|    clip_fraction        | 0.107       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.02       |
|    explained_variance   | -9.54e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 63.9        |
|    n_updates            | 300         |
|    policy_gradient_loss | -0.0138     |
|    value_loss           | 128         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.1   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9.06        |
|    ep_rew_mean          | -40.9       |
| time/                   |             |
|    fps                  | 993         |
|    iterations           | 41          |
|    time_elapsed         | 84          |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.011049003 |
|    clip_fraction        | 0.129       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.75       |
|    explained_variance   | -2.38e-07   |
|    learning_rate        | 0.0003      |
|    loss                 | 46.5        |
|    n_updates            | 400         |
|    policy_gradient_loss | -0.0106     |
|    value_loss           | 114         |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 9.04

In [28]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 5, 2, 8, 6, 9, 3, 4, 1, 7], g = 34.55


In [27]:
x_min = []
g_min = 100000000
print(datetime.datetime.now())
for i, x in enumerate(itertools.permutations(range(1,N), N-1)):
    x = (0,) + x
    g = np.sum(M[x[:-1], x[1:]])+M[x[-1], x[0]]
    if g < g_min:
        g_min = g
        x_min = x
    if i % 100000 == 0:
        print(i, x_min, g_min)
print(datetime.datetime.now())
print('Optimal solution:')
print(x_min, g_min)

2022-03-13 19:58:01.039612
0 (0, 1, 2, 3, 4, 5, 6, 7, 8, 9) 54.75999999999999
100000 (0, 3, 4, 1, 7, 5, 2, 6, 9, 8) 36.46
200000 (0, 5, 2, 8, 6, 9, 3, 4, 1, 7) 34.55
300000 (0, 5, 2, 8, 6, 9, 3, 4, 1, 7) 34.55
2022-03-13 19:58:06.565740
Optimal solution:
(0, 5, 2, 8, 6, 9, 3, 4, 1, 7) 34.55


In [32]:
ray.shutdown()
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:63062',
 'object_store_address': '/tmp/ray/session_2022-03-13_20-03-32_400944_23142/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-03-13_20-03-32_400944_23142/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-03-13_20-03-32_400944_23142',
 'metrics_export_port': 64725,
 'gcs_address': '127.0.0.1:59736',
 'node_id': '6bad5deaf6db8bf243f707c7d8fedd276054729843fc4a681c988362'}

In [33]:
agent = rllib.agents.ppo.PPOTrainer(config=config, env=MyEnv)

2022-03-13 20:03:45,078	WARNING util.py:55 -- Install gputil for GPU system monitoring.
(RolloutWorker pid=23753) 2022-03-13 20:03:45,057	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!


In [34]:
print(datetime.datetime.now())
for i in range(31):
   # Perform one iteration of training the policy with PPO
   result = agent.train()
   if i % 10 == 0:
       #print(pretty_print(result))
       print('i: ', i)
       print('mean episode length:', result['episode_len_mean'])
       print('max episode reward:', result['episode_reward_max'])
       print('mean episode reward:', result['episode_reward_mean'])
       print('min episode reward:', result['episode_reward_min'])
       print('total episodes:', result['episodes_total'])
       print()

       checkpoint = agent.save()
print(datetime.datetime.now())

2022-03-13 20:03:46.564128
i:  0
mean episode length: 25.35031847133758
max episode reward: -72.99000000000001
mean episode reward: -214.08828025477706
min episode reward: -661.95
total episodes: 157

i:  10
mean episode length: 9.2830626450116
max episode reward: -37.42
mean episode reward: -49.96266821345708
min episode reward: -81.33999999999997
total episodes: 3438

i:  20
mean episode length: 9.018018018018019
max episode reward: -34.95
mean episode reward: -41.2081981981982
min episode reward: -58.39
total episodes: 7849

i:  30
mean episode length: 9.00900900900901
max episode reward: -34.55
mean episode reward: -38.070923423423416
min episode reward: -53.459999999999994
total episodes: 12286

2022-03-13 20:09:12.743320


In [44]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action = agent.compute_single_action(obs, explore = False)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 3, 6, 9, 8, 1, 7, 5, 4, 2], g = 35.96


In [50]:
def env_creator(env_config):
    return MyEnv(env_config)  # return an env instance

register_env("my_env", env_creator)

In [60]:
experiment = tune.run(
    rllib.agents.ppo.PPOTrainer,
    config={
        "env": "my_env",
        "num_gpus": 0,
        "num_workers": 1,
        "framework": "torch"
    },
    metric="episode_reward_mean",
    mode="max",
    stop={"training_iteration": 50},
    checkpoint_at_end=True
)

Trial name,status,loc
PPOTrainer_my_env_246f0_00000,PENDING,


(PPOTrainer pid=25313) 2022-03-13 20:58:23,620	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=25313) 2022-03-13 20:58:23,621	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=25312) 2022-03-13 20:58:33,208	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!


Trial name,status,loc
PPOTrainer_my_env_246f0_00000,RUNNING,127.0.0.1:25313


(PPOTrainer pid=25313) 2022-03-13 20:58:33,269	WARNING util.py:55 -- Install gputil for GPU system monitoring.


Trial name,status,loc
PPOTrainer_my_env_246f0_00000,RUNNING,127.0.0.1:25313


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_246f0_00000,RUNNING,127.0.0.1:25313,49,514.588,196000,-37.5498,-34.55,-52.41,9.0045


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_246f0_00000,RUNNING,127.0.0.1:25313,49,514.588,196000,-37.5498,-34.55,-52.41,9.0045


Result for PPOTrainer_my_env_246f0_00000:
  agent_timesteps_total: 200000
  custom_metrics: {}
  date: 2022-03-13_21-07-20
  done: true
  episode_len_mean: 9.006756756756756
  episode_media: {}
  episode_reward_max: -34.55
  episode_reward_mean: -37.12144144144145
  episode_reward_min: -55.96000000000001
  episodes_this_iter: 444
  episodes_total: 20816
  experiment_id: b01739d93d044cc4b13cba163f16684d
  hostname: MacBook-Pro-Vladimir.local
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 0.5062499999999999
          cur_lr: 5.0000000000000016e-05
          entropy: 0.37140962946158584
          entropy_coeff: 0.0
          kl: 0.006195005703386621
          policy_loss: -0.016394468080452692
          total_loss: 93.04049556280977
          vf_explained_var: 1.922730476625504e-09
          vf_loss: 93.05375423636488
        model: {}
    num_agent_steps_sampled: 200000
    num_agent_st

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_my_env_246f0_00000,TERMINATED,127.0.0.1:25313,50,526.16,200000,-37.1214,-34.55,-55.96,9.00676


(PPOTrainer pid=25313) 2022-03-13 21:07:20,700	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!
2022-03-13 21:07:21,259	INFO tune.py:636 -- Total run time: 551.92 seconds (551.35 seconds for the tuning loop).


In [63]:
agent.restore(experiment.best_checkpoint)

2022-03-13 21:10:13,767	INFO trainable.py:472 -- Restored on 127.0.0.1 from checkpoint: /Users/vladimirsudakov/ray_results/PPOTrainer_2022-03-13_20-58-09/PPOTrainer_my_env_246f0_00000_0_2022-03-13_20-58-09/checkpoint_000050/checkpoint-50
2022-03-13 21:10:13,769	INFO trainable.py:480 -- Current state after restoring: {'_iteration': 50, '_timesteps_total': 200000, '_time_total': 526.1604516506195, '_episodes_total': 20816}


In [64]:
obs = env.reset()
g = 0
actions = [0]
for i in range(1000):
    action = agent.compute_single_action(obs, explore = False)
    obs, reward, done, info = env.step(action)
    g += M[actions[-1], action + 1]
    actions.append(action + 1)
    if done:
        obs = env.reset()
        g += M[actions[-1], 0]
        print(f"{actions}, g = {g}")
        break
env.close()

[0, 5, 2, 8, 6, 9, 3, 4, 1, 7], g = 34.55
